# Custom exercice: Using TAP to get usable tables for LePHARE

**Learning objective:** Create 2 tables of LSST objects: one for galaxies, one for stars

**LSST data products:** `Object` table

**Packages:** `lsst.rsp.get_tap_service`

## 1. Introduction

TAP (Table Access Protocol) provides standardized access to catalog data for discovery, search, and retrieval.
Full [documentation for TAP](http://www.ivoa.net/documents/TAP) is provided by the
International Virtual Observatory Alliance (IVOA).

### 1.1. TAP glossary

* **schema** - Database terminology for the abstract design that represents the storage of data in a database.
* **TAP schema** - The set of tables that describe the data tables and their columns.
* **table collection** - A collection of tables (catalogs), e.g., for a given data release.
* **table** - A collection of related data held in a table format in a database. 
* **query** - A string formatted in ADQL that selects data from a table, with contraints if desired. 
* **results** - The output of the TAP service's search method when a query is passed.


### 1.2. Reminder: ADQL statements

The [documentation for ADQL](http://www.ivoa.net/documents/latest/ADQL.html)
includes more information about syntax, keywords, operators, functions, and so on.
ADQL is similar to SQL (Structured Query Langage).

A typical ADQL statement has at least three components:

```
SELECT <columns> FROM <catalog> WHERE <constraints>
```

where
 * `<columns>` is a comma-separated list of the columns to return,
 * `<catalog>` is the name of the catalog to retreive data from, and
 * `<constraints>` imposes a restriction that only rows with column values that meet the constraints are returned.

For example, say there is a catalog called "mysurveydata" with 5 columns, "col1", "col2", and so on.
The ADQL statement:

```
SELECT col3, col4, col5 FROM mysurveydata WHERE col1 > 0.5 AND col5 < 10
```

would 
return a table that has three columns, and as many rows as meet both of the restrictions in the `WHERE` statement.

### 1.3. Import packages


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import lsst.geom as geom

from lsst.rsp import get_tap_service
from lsst.rsp.utils import get_pyvo_auth
from lsst.rsp.service import get_siav2_service


## 2. Initialisation of the query

We will be looking at the DP1 catalog schema table to pick columns that would be helpful to identify the source-type. Then we will make our final query to get the right data.

### 2.1 First look at the schema tables.

In [ ]:
service = get_tap_service("tap")
assert service is not None

In [ ]:
results = service.search('SELECT * FROM tap_schema.schemas')

In [ ]:
results.to_table()

In [ ]:
del results

We will obviously chose dp1, but we need to choose from which catalog we will retrieve data. Thus we look at the tap_schema.tables category.

In [ ]:
query = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = 'dp1'" \
        "ORDER BY table_index ASC"
results = service.search(query).to_table()
results

In [ ]:
del query, results

Lets is use the object catalog for this time.

### 2.2 Pick right columns

In the dp1.Object catalog, we need to identify which columns should be useful for LePHARE and later galax

In [ ]:
query = "SELECT column_name, datatype, description, unit " \
        "FROM tap_schema.columns " \
        "WHERE table_name = 'dp1.Object'"
results = service.search(query).to_table()
results

In [ ]:
search_string = 'ModelMag' #g_cModelMag, g_cModelMagErr
# search_string = 'g_psfFlux'
for cname in results['column_name']:
    if cname.find(search_string) > -1:
        print(cname)

In [ ]:
del results

len(results) = 1296 --> we won't need every row

### 2.3. Reminder: useful keywords

We will use a few useful category of keywords to retrieve particular data type:
 * `coord`: ra, dec, errors. Coordinates of the object.
 * `(de)blend`: blendness category includes flag characterizing the mixing of several sources. Perhaps it should be taken into account if blendness and extendedness appeared both as True for a single object
 * `magnitude`: magnitude categories include values associated to ugrizy mag, like numerical value, errors and extendedness. 
 * `{band}_extendedness` where band = ugrizy. This is probably the most useful keywords to identify the source or object type (galaxy or star)

## 3. Trial in the ecdfs region

In [ ]:
target_ra = 53.195
target_dec = -27.703
target_r = 1

We won't take into account the AGNs (present in the DIAobject catalog). We will only look at galaxies and stars in the object catalog. To differentiate them efficiently within the LSST catalog, we establish a selection criterion : if 3 bands or more show {band}_extendedness = 1, we'll consider it as a galaxies. Unless the sources will be flagged as stars. But that something that will be useful later.

### 3.1 Full useful parameters table

The first table include every parameters that could be useful in star-galaxies separation, not differentiating the source types.

In [ ]:
query = "SELECT objectId, coord_ra, coord_dec, " + \
        "u_cModelMag, u_cModelMagErr, u_extendedness, " + \
        "g_cModelMag, g_cModelMagErr, g_extendedness, " + \
        "r_cModelMag, r_cModelMagErr, r_extendedness, " + \
        "i_cModelMag, i_cModelMagErr, i_extendedness, " + \
        "z_cModelMag, z_cModelMagErr, z_extendedness, " + \
        "y_cModelMag, y_cModelMagErr, y_extendedness, refBand " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "


In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

### 3.2 Galaxy table

This is already in the short MEME lephare format.

In [ ]:
query = "SELECT objectId, " + \
        "u_cModelMag, u_cModelMagErr, " + \
        "g_cModelMag, g_cModelMagErr, " + \
        "r_cModelMag, r_cModelMagErr, " + \
        "i_cModelMag, i_cModelMagErr, " + \
        "z_cModelMag, z_cModelMagErr, " + \
        "y_cModelMag, y_cModelMagErr " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "(g_extendedness = 1) AND " + \
        "(r_extendedness = 1) AND " + \
        "(i_extendedness = 1) AND " + \
        "(z_extendedness = 1) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

In [ ]:
job.delete()
del query, results, tab

### 3.2 Star table

In [ ]:
query = "SELECT objectId, " + \
        "u_cModelMag, u_cModelMagErr, " + \
        "g_cModelMag, g_cModelMagErr, " + \
        "r_cModelMag, r_cModelMagErr, " + \
        "i_cModelMag, i_cModelMagErr, " + \
        "z_cModelMag, z_cModelMagErr, " + \
        "y_cModelMag, y_cModelMagErr " + \
        "FROM dp1.Object AS obj " + \
        "WHERE (i_cModelFlux/i_cModelFluxErr > 15) AND " + \
        "(u_cModelFlux/u_cModelFluxErr > 5) AND " + \
        "(shape_flag = 0) AND " + \
        "(g_extendedness = 0) AND " + \
        "(r_extendedness = 0) AND " + \
        "(i_extendedness = 0) AND " + \
        "(z_extendedness = 0) AND " + \
        "CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS',"+str(target_ra)+","+str(target_dec)+", "+str(target_r)+")) = 1 "



In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result()
tab = results.to_table()
tab

In [ ]:
job.delete()
del query, results, tab

## DC2

In [ ]:
query = "SELECT mt.id_truth_type AS mt_id_truth_type, "\
        "mt.match_objectId AS mt_match_objectId, "\
        "ts.ra AS ts_ra, "\
        "ts.dec AS ts_dec, "\
        "ts.truth_type AS ts_truth_type, "\
        "ts.is_pointsource AS ts_is_pointsource, "\
        "ts.redshift AS ts_redshift, "\
        "obj.coord_ra AS obj_coord_ra, "\
        "obj.coord_dec AS obj_coord_dec, "\
        "obj.refExtendedness AS obj_refExtendedness, "\
        "scisql_nanojanskyToAbMag(obj.u_cModelFlux) AS umag, "\
        "scisql_nanojanskyToAbMag(obj.g_cModelFlux) AS gmag, "\
        "scisql_nanojanskyToAbMag(obj.r_cModelFlux) AS rmag, "\
        "scisql_nanojanskyToAbMag(obj.i_cModelFlux) AS imag, "\
        "scisql_nanojanskyToAbMag(obj.z_cModelFlux) AS zmag, "\
        "scisql_nanojanskyToAbMag(obj.y_cModelFlux) AS ymag, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.u_cModelFluxErr) AS umag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.g_cModelFluxErr) AS gmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.r_cModelFluxErr) AS rmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.i_cModelFluxErr) AS imag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.z_cModelFluxErr) AS zmag_err, "\
        "scisql_nanojanskyToAbMagSigma(obj.g_cModelFlux, obj.y_cModelFluxErr) AS ymag_err "\
        "FROM dp02_dc2_catalogs.MatchesTruth AS mt "\
        "JOIN dp02_dc2_catalogs.TruthSummary AS ts ON mt.id_truth_type = ts.id_truth_type "\
        "JOIN dp02_dc2_catalogs.Object AS obj ON mt.match_objectId = obj.objectId "\
        "WHERE CONTAINS(POINT('ICRS', obj.coord_ra, obj.coord_dec), CIRCLE('ICRS', 62.0, -37.0, 0.5)) = 1 "\
        "AND scisql_nanojanskyToAbMag(obj.y_cModelFlux)/scisql_nanojanskyToAbMagSigma(obj.y_cModelFlux, obj.y_cModelFluxErr) > 5 "\
        "AND obj.detect_isPrimary = 1"

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table().to_pandas()
print(len(results))

In [ ]:
results